In [ ]:
import sys
import pandas as pd 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet
from quickbin import bin2d

from photutils.background import Background2D, MedianBackground
from astropy.stats import SigmaClip

In [ ]:
# make a list of fuv to make / look at 

og_aspect = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet').to_pandas()
og_aspect = og_aspect.dropna(subset=['fuv_temp'])

og_aspect = og_aspect[og_aspect['legs'] == 0]

sampled_eclipses = og_aspect.sample(n=200, random_state=5)
sampled_eclipses

In [ ]:
for index, eclipse in sampled_eclipses['eclipse'].items():
    print(f"running {eclipse}")
    try: 
        execute_pipeline(
                int(eclipse),
                "NUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/media/bekah/BekahA/background_test/pics_and_photonlists",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": True},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        
    except Exception as e: 
        print(e)
